# Study of urban green space in relation to happiness

## Problem Description

### Assisting the urban planning department of Brussels, Belgium to explore the density of urban green space and location for new green space.

A study of the University of Vermont shows a clear relationship between urban green space and happiness. The study was conducted using sentiment analysis (Hedonometer) on Twitter posts.
This allowed measurement of the level of happiness or positiveness by comparing the sentiment at the time before visiting, during visiting and after visiting urban parks.


Abstract from the study:
1. With more people living in cities, we are witnessing a decline in exposure to nature. A growing body of research has demonstrated an association between nature contact and improved mood.
2. Here, we used Twitter and the Hedonometer, a world analysis tool, to investigate how sentiment, or the estimated happiness of the words people write, varied before, during, and after visits to San Francisco's urban park system. We found that sentiment was substantially higher during park visits and remained elevated for several hours following the visit.
3. Leveraging differences in vegetative cover across park types, we explored how different types of outdoor public spaces may contribute to subjective well‐being. Tweets during visits to Regional Parks, which are greener and have greater vegetative cover, exhibited larger increases in sentiment than tweets during visits to Civic Plazas and Squares.
4. Finally, we analyed word frequencies to explore several mechanisms theorized to link nature exposure with mental and cognitive benefits. Negation words such as ‘no’, ‘not’, and ‘don't’ decreased in frequency during visits to urban parks.
5. These results can be used by urban planners and public health officials to better target nature contact recommendations for growing urban populations.

This is not the only study which has shown that having access to, or exposure to nature makes people kinder, happier and stimulates creativity. 

### Audience
The study is conducted for the urban planning department of the city of Brussels. This will allow them to locate areas for new green space development in the city. As such they can more effectively use their available budgets.

The results of the study can potentially benefit all residents of the city of Brussels.

Mental health is an important topic worldwide. With an ever increasing healthcare cost, it is important to explore natural and non-medical factors which can have a substantial benefit on the health of the general public. It is far more important to prevent then to cure.
Taking the above into account, we will attempt to use maps and location data to visualize the balance between greenspace and urban space in different parts of the city. Just as we are building further on a previous data science study based on sentiment analysis, this analysis can then in turn be used later on to link to public health records to map if there is indeed a link between mental health and happiness and relative access to urban greenspace.


References:

https://www.uvm.edu/uvmnews/news/city-parks-lift-mood-much-christmas-twitter-study-shows

https://besjournals.onlinelibrary.wiley.com/doi/full/10.1002/pan3.10045

https://greatergood.berkeley.edu/article/item/how_nature_makes_you_kinder_happier_more_creative



## Data Description
The following data sources can be leveraged to perform this study:

**Foursquare:**

Parks in Brussels:

https://foursquare.com/explore?mode=url&near=Brussels%2C%20Belgium&nearGeoId=72057594040728802&q=Park

Forest in Brussels:

https://foursquare.com/explore?mode=url&near=Brussels%2C%20Belgium&nearGeoId=72057594040728802&q=forest

The above links are a guideline to initiate the study and will be translated in API queries when performing the actual data analysis using the Foursquare Developer API. In case we find that Brussels has a near perfect distribution of parks, we can leverage the average rating values from Foursquare to decide which parks need changes, maintenance or redevelopment.

**Foursquare data items which can potentially be used:**
- Location name for identification
- Latitude and longitude for mapping
- Rating of the location for quality indication
- Amount of tips to estimate popularity

**Open Data Brussels:**
Parks:
https://opendata.brussels.be/explore/dataset/parks/information/

Parks data from the Open Data Brussels platform can perhaps be used during the analysis to see how good the Foursquare coverage is. Based on this the city council could perhaps leverage the use of social internet tools like Foursquare to assure that  residents and tourists can get easy access to info regarding urban green space.

**Open Data Brussels data items which can potentially be used:**
- Location name for identification
- Latitude and longitude for mapping
